In [1]:
import torch
print(torch.__version__)
print(torch.cuda.device_count())
print(torch.cuda.is_available())

1.10.1+cu102
1
True


In [2]:
# Importing libraries

import os
import cv2
import torch
import torch.nn as nn
import torch.optim as optim
import torchvision
import torchvision.transforms as transforms
import torch.nn.functional as F

import numpy as np
import pandas as pd

from torch.utils import data
from torch.utils.data import DataLoader
from torch.optim.lr_scheduler import MultiStepLR
import matplotlib.pyplot as plt

from sklearn.metrics import mean_squared_error

C:\Users\Lakshmikar Reddy\anaconda3\envs\gpu\lib\site-packages\torchvision\io\image.py:11: UserWarning: Failed to load image Python extension: [WinError 126] The specified module could not be found
  warn(f"Failed to load image Python extension: {e}")


In [3]:
pwd

'D:\\MS_AI_YU\\Fall 2022\\CV\\Project for Conference\\My CNN model'

In [4]:
dataroot = "./data/dataset/"
valroot = "./valid/"
ckptroot = "./"

lr = 1e-4
weight_decay = 1e-5
batch_size = 32
num_workers = 0
#test_size = 0.8
shuffle = True

epochs = 10
start_epoch = 0
resume = False

In [5]:
# Path for test data set
testroot = "./test/"

In [6]:
# Helper functions

def toDevice(datas, device):
    """Enable cuda."""
    imgs, angles = datas
    return imgs.float().to(device), angles.float().to(device)


# def augment(dataroot, imgName, angle):
#     """Data augmentation."""
#     name = dataroot + 'IMG/' + imgName.split('\\')[-1]
#     current_image = cv2.imread(name)
#     #current_image = cv2.cvtColor(current_image,cv2.COLOR_RGB2YUV)
#     #current_image = cv2.resize(current_image, (128,128))

#     if current_image is None:
#         print(name)

#     current_image = current_image[65:-25, :, :]
#     if np.random.rand() < 0.5:
#         current_image = cv2.flip(current_image, 1)
#         angle = angle * -1.0
#     #print(current_image, angle)
#     return current_image, angle

def augment(dataroot, imgName, angle):
    """Data augmentation."""
    name = dataroot + 'IMG/' + imgName.split('\\')[-1]
    current_image = cv2.imread(name)
    current_image = current_image[65:-25, :, :]
    
    current_image = cv2.cvtColor(current_image,cv2.COLOR_RGB2YUV)

    if np.random.rand() < 0.5:
        current_image = cv2.flip(current_image, 1)
        angle = angle * -1.0
    #print(current_image, angle)
    return current_image, angle

In [7]:
# Helper functions for reading test data set

def readtestdata(testroot, imgName, angle):
    """Reading testdata."""
    name = testroot + 'IMG/' + imgName.split('\\')[-1]
    current_image = cv2.imread(name)
    
    if current_image is None:
        print(name)

    current_image = current_image[65:-25, :, :]
#     if np.random.rand() < 0.5:
#         current_image = cv2.flip(current_image, 1)
#         angle = angle * -1.0
    #print(current_image, angle)
    return current_image, angle

In [8]:
# # Data Loading
# def load_data(data_dir, test_size):
#     """Load training data and train validation split"""
#     pass

#     # reads CSV file into a single dataframe variable
#     data_df = pd.read_csv(os.path.join(data_dir, 'driving_log.csv'),
#                           names=['center', 'left', 'right', 'steering', 'throttle', 'reverse', 'speed'])

#     # Divide the data into training set and validation set
#     train_len = int(test_size * data_df.shape[0])
#     valid_len = data_df.shape[0] - train_len
#     trainset, valset = data.random_split(
#         data_df.values.tolist(), lengths=[train_len, valid_len])

#     return trainset, valset

# trainset, valset = load_data(dataroot, test_size)
# type(trainset)

In [9]:
# Data Loading for training and validation sets
trainset = pd.read_csv(os.path.join(dataroot, 'driving_log.csv'),
                          names=['center', 'left', 'right', 'steering', 'throttle', 'reverse', 'speed']).values.tolist()
valset = pd.read_csv(os.path.join(valroot, 'driving_log.csv'),
                          names=['center', 'left', 'right', 'steering', 'throttle', 'reverse', 'speed']).values.tolist()

In [10]:
#Test Data Loading
testset = pd.read_csv(os.path.join(testroot, 'driving_log.csv'),
                          names=['center', 'left', 'right', 'steering', 'throttle', 'reverse', 'speed']).values.tolist()

In [11]:

# Creating dataset

class TripletDataset(data.Dataset):

    def __init__(self, dataroot, samples, transform=None):
        self.samples = samples
        self.dataroot = dataroot
        self.transform = transform

    def __getitem__(self, index):
        batch_samples = self.samples[index]
        steering_angle = float(batch_samples[3])

        center_img, steering_angle_center = augment(self.dataroot, batch_samples[0], steering_angle)
        left_img, steering_angle_left     = augment(self.dataroot, batch_samples[1], steering_angle + 0.4)
        right_img, steering_angle_right   = augment(self.dataroot, batch_samples[2], steering_angle - 0.4)

        center_img = self.transform(center_img)
        left_img   = self.transform(left_img)
        right_img  = self.transform(right_img)
        
        center_img = center_img.transpose((2, 0, 1))
        left_img = left_img.transpose((2, 0, 1))
        right_img = right_img.transpose((2, 0, 1))

        #print(center_img.shape)

        return (center_img, steering_angle_center), (left_img, steering_angle_left), (right_img, steering_angle_right)

    def __len__(self):
        return len(self.samples)

In [12]:
# Creating test datset

class TestDataset(data.Dataset):

    def __init__(self, testroot, samples, transform=None):
        self.samples = samples
        self.testroot = testroot
        self.transform = transform

    def __getitem__(self, index):
        batch_samples = self.samples[index]
        steering_angle = float(batch_samples[3])

        center_img, steering_angle_center = readtestdata(self.testroot, batch_samples[0], steering_angle)
        left_img, steering_angle_left     = readtestdata(self.testroot, batch_samples[1], steering_angle + 0.4)
        right_img, steering_angle_right   = readtestdata(self.testroot, batch_samples[2], steering_angle - 0.4)

        center_img = self.transform(center_img)
        left_img   = self.transform(left_img)
        right_img  = self.transform(right_img)
        
        center_img = center_img.transpose((2, 0, 1))
        left_img = left_img.transpose((2, 0, 1))
        right_img = right_img.transpose((2, 0, 1))

        #print(center_img.shape)

        return (center_img, steering_angle_center), (left_img, steering_angle_left), (right_img, steering_angle_right)

    def __len__(self):
        return len(self.samples)

In [13]:
# Data Loader

print("==> Preparing dataset ...")
def data_loader(dataroot, valroot, trainset, valset, batch_size, shuffle, num_workers):
    """Self-Driving vehicles simulator dataset Loader.

    Args:
        trainset: training set
        valset: validation set
        batch_size: training set input batch size
        shuffle: whether shuffle during training process
        num_workers: number of workers in DataLoader

    Returns:
        trainloader (torch.utils.data.DataLoader): DataLoader for training set
        testloader (torch.utils.data.DataLoader): DataLoader for validation set
    """
    transformations = transforms.Compose(
        [transforms.Lambda(lambda x: (x / 127.5) - 1.0)])

    # Load training data and validation data
    training_set = TripletDataset(dataroot, trainset, transformations)
    trainloader = DataLoader(training_set,
                             batch_size=batch_size,
                             shuffle=shuffle,
                             num_workers=num_workers)

    validation_set = TripletDataset(valroot, valset, transformations)
    valloader = DataLoader(validation_set,
                           batch_size=batch_size,
                           shuffle=shuffle,
                           num_workers=num_workers)

    return trainloader, valloader


trainloader, validationloader = data_loader(dataroot, valroot,
                                            trainset, valset,
                                            batch_size,
                                            shuffle,
                                            num_workers)

==> Preparing dataset ...


In [14]:
# Data loader for test dataset
batch_size = 40
def testdata_loader(testroot, testset, batch_size, num_workers):
    """Self-Driving vehicles simulator dataset Loader.

    Args:
        testset: training set
        batch_size: test set input batch size
        num_workers: number of workers in DataLoader

    Returns:
        testloader (torch.utils.data.DataLoader): DataLoader for test set
     """
    transformations = transforms.Compose(
        [transforms.Lambda(lambda x: (x / 127.5) - 1.0)])

    # Load test data
    test_set = TestDataset(testroot, testset, transformations)
    #print(test_set.shape())
    testloader = DataLoader(test_set,
                             batch_size=batch_size,
                             num_workers=num_workers)
    return testloader


testloader = testdata_loader(testroot,
                             testset,
                             batch_size,
                             num_workers)
#print(testloader)

In [15]:
#Creating model
# CNN1 - 3x3 filters, 7 convolution layers

# class ConvNet(nn.Module):
#     def __init__(self):
#         super(ConvNet, self).__init__()
#         # Input image size = 70x320x3

#         self.conv1 = nn.Conv2d(3, 16, 3)  # 68x318
#         self.conv2 = nn.Conv2d(16, 32, 3) # 66x316
#         self.pool1 = nn.MaxPool2d(2, 2) # 33x158

#         self.conv3 = nn.Conv2d(32, 64, 3) # 31x156
#         self.conv4 = nn.Conv2d(64, 64, 3) # 29x154
#         self.pool2 = nn.MaxPool2d(2, 2) # 14x77

#         self.conv5 = nn.Conv2d(64, 64, 3) # 12x75
#         self.conv6 = nn.Conv2d(64, 64, 3) # 10x73
#         self.conv7 = nn.Conv2d(64, 64, 3) # 8x71
#         self.pool3 = nn.MaxPool2d(2, 2) # 4x35

        
#         self.fc1 = nn.Linear(4*35*64, 128)
#         self.dropout1 = nn.Dropout2d(0.25)
#         self.fc2 = nn.Linear(128, 64)
#         self.dropout2 = nn.Dropout2d(0.5)
#         self.fc3 = nn.Linear(64, 1)
        

#     def forward(self, x):
  
#         x = F.relu(self.conv1(x))
#         x = self.pool1(F.relu(self.conv2(x)))
#         x = F.relu(self.conv3(x))
#         x = self.pool2(F.relu(self.conv4(x)))
#         x = F.elu(self.conv5(x))
#         x = F.elu(self.conv6(x))
#         x = self.pool3(F.relu(self.conv7(x)))

#         x = x.view(-1, 4*35*64)
#         x = F.relu(self.fc1(x))
#         #print(x.shape)
#         x = self.dropout1(x)
#         x = F.relu(self.fc2(x))
#         x = self.dropout2(x)
#         x = self.fc3(x)
#         return x

# # Define model
# print("==> Initialize model ...")
# model = ConvNet()
# print("==> Initialize model done ...")

In [16]:
# # Creating model
#CNN2 - 6 convolution layers, 3x3 filters, padding-same

# class ConvNet(nn.Module):
#     def __init__(self):
#         super(ConvNet, self).__init__()
#         # Input image size = 70x320x3

#         self.conv1 = nn.Conv2d(3, 16, 3, padding = 'same')  # 70x320
#         self.conv2 = nn.Conv2d(16, 32, 3, padding = 'same') # 70x320
#         self.pool1 = nn.MaxPool2d(2, 2) # 35x160

#         self.conv3 = nn.Conv2d(32, 64, 3, padding = 'same') # 35x160
#         self.conv4 = nn.Conv2d(64, 64, 3, padding = 'same') # 35x160
#         self.pool2 = nn.MaxPool2d(2, 2) # 17x80

#         self.conv5 = nn.Conv2d(64, 64, 3, padding = 'same') # 17x80
#         self.conv6 = nn.Conv2d(64, 64, 3, padding = 'same') # 17x80
#         self.pool3 = nn.MaxPool2d(2, 2) # 8x40

        
#         self.fc1 = nn.Linear(8*40*64, 128)
#         self.dropout1 = nn.Dropout2d(0.25)
#         self.fc2 = nn.Linear(128, 64)
#         self.dropout2 = nn.Dropout2d(0.5)
#         self.fc3 = nn.Linear(64, 1)
        

#     def forward(self, x):
  
#         x = F.relu(self.conv1(x))
#         x = self.pool1(F.relu(self.conv2(x)))
#         x = F.relu(self.conv3(x))
#         x = self.pool2(F.relu(self.conv4(x)))
#         x = F.relu(self.conv5(x))
#         x = self.pool3(F.relu(self.conv6(x)))

#         x = x.view(-1, 8*40*64)
#         x = F.relu(self.fc1(x))
#         #print(x.shape)
#         x = self.dropout1(x)
#         x = F.relu(self.fc2(x))
#         x = self.dropout2(x)
#         x = self.fc3(x)
#         return x

# # Define model
# print("==> Initialize model ...")
# model = ConvNet()
# print("==> Initialize model done ...")

In [17]:
# Creating model
# CNN3 - 3 convolution layers, 3x3 filters, padding-same

# class ConvNet(nn.Module):
#     def __init__(self):
#         super(ConvNet, self).__init__()
#         # Input image size = 70x320x3

#         self.conv1 = nn.Conv2d(3, 32, 3, padding = 'same')  # 70x320
#         self.pool1 = nn.MaxPool2d(2, 2) # 35x160

#         self.conv2 = nn.Conv2d(32, 64, 3, padding = 'same') # 35x160
#         self.pool2 = nn.MaxPool2d(2, 2) # 17x80

#         self.conv3 = nn.Conv2d(64, 128, 3, padding = 'same') # 17x80
#         self.pool3 = nn.MaxPool2d(2, 2) # 8x40
#         self.dropout1 = nn.Dropout2d(0.25)

        
#         self.fc1 = nn.Linear(8*40*128, 512)
#         self.dropout2 = nn.Dropout2d(0.25)
#         self.fc2 = nn.Linear(512, 1)
        

#     def forward(self, x):
  
#         x = self.pool1(F.relu(self.conv1(x)))
#         x = self.pool2(F.relu(self.conv2(x)))
#         x = self.pool3(F.relu(self.conv3(x)))
#         x = self.dropout1(x)

#         x = x.view(-1, 8*40*128)
#         x = F.relu(self.fc1(x))
#         x = self.dropout2(x)
#         x = self.fc2(x)
#         return x

# # Define model
# print("==> Initialize model ...")
# model = ConvNet()
# print("==> Initialize model done ...")

In [18]:
# Creating model
# CNN4 - 5 convolution layers, 5x5 filters

# class ConvNet(nn.Module):
#     def __init__(self):
#         super(ConvNet, self).__init__()
#         # Input image size = 70x320x3

#         self.conv1 = nn.Conv2d(3, 32, 5, padding = 'same')  # 70x320
#         self.pool1 = nn.MaxPool2d(2, 2) # 35x160

#         self.conv2 = nn.Conv2d(32, 64, 5, padding = 'same') # 35x160
#         self.pool2 = nn.MaxPool2d(2, 2) # 17x80

#         self.conv3 = nn.Conv2d(64, 128, 5, padding = 'same') # 17x80
#         self.pool3 = nn.MaxPool2d(2, 2) # 8x40
        
#         self.conv4 = nn.Conv2d(128, 128, 5, padding = 'same') # 8x40
#         self.pool4 = nn.MaxPool2d(2, 2) # 4x20
        
#         self.conv5 = nn.Conv2d(128, 128, 5, padding = 'same') # 4x20
#         self.pool5 = nn.MaxPool2d(2, 2) # 2x10
        
#         self.dropout1 = nn.Dropout2d(0.25)

        
#         self.fc1 = nn.Linear(2*10*128, 512)
#         self.dropout2 = nn.Dropout2d(0.25)
#         self.fc2 = nn.Linear(512, 1)
        

#     def forward(self, x):
  
#         x = self.pool1(F.relu(self.conv1(x)))
#         x = self.pool2(F.relu(self.conv2(x)))
#         x = self.pool3(F.relu(self.conv3(x)))
#         x = self.pool4(F.relu(self.conv4(x)))
#         x = self.pool5(F.relu(self.conv5(x)))
#         x = self.dropout1(x)

#         x = x.view(-1, 2*10*128)
#         x = F.relu(self.fc1(x))
#         x = self.dropout2(x)
#         x = self.fc2(x)
#         return x

# # Define model
# print("==> Initialize model ...")
# model = ConvNet()
# print("==> Initialize model done ...")

In [19]:
# # Creating model
# # CNN5 - 5 convolution layers, 7x7 and 5x5 filters

# class ConvNet(nn.Module):
#     def __init__(self):
#         super(ConvNet, self).__init__()
#         # Input image size = 70x320x3

#         self.conv1 = nn.Conv2d(3, 32, 7, padding = 'same')  # 70x320
#         self.pool1 = nn.MaxPool2d(2, 2) # 35x160

#         self.conv2 = nn.Conv2d(32, 64, 7, padding = 'same') # 35x160
#         self.pool2 = nn.MaxPool2d(2, 2) # 17x80

#         self.conv3 = nn.Conv2d(64, 128, 7, padding = 'same') # 17x80
#         self.pool3 = nn.MaxPool2d(2, 2) # 8x40
        
#         self.conv4 = nn.Conv2d(128, 128, 5, padding = 'same') # 8x40
#         self.pool4 = nn.MaxPool2d(2, 2) # 4x20
        
#         self.conv5 = nn.Conv2d(128, 128, 5, padding = 'same') # 4x20
#         self.pool5 = nn.MaxPool2d(2, 2) # 2x10
        
#         self.dropout1 = nn.Dropout2d(0.25)

        
#         self.fc1 = nn.Linear(2*10*128, 512)
#         self.dropout2 = nn.Dropout2d(0.25)
#         self.fc2 = nn.Linear(512, 1)
        

#     def forward(self, x):
  
#         x = self.pool1(F.relu(self.conv1(x)))
#         x = self.pool2(F.relu(self.conv2(x)))
#         x = self.pool3(F.relu(self.conv3(x)))
#         x = self.pool4(F.relu(self.conv4(x)))
#         x = self.pool5(F.relu(self.conv5(x)))
#         x = self.dropout1(x)

#         x = x.view(-1, 2*10*128)
#         x = F.relu(self.fc1(x))
#         x = self.dropout2(x)
#         x = self.fc2(x)
#         return x

# # Define model
# print("==> Initialize model ...")
# model = ConvNet()
# print("==> Initialize model done ...")

In [20]:
# # Creating model
# # CNN6 - 3 convolution layers, 7x7 filters

# class ConvNet(nn.Module):
#     def __init__(self):
#         super(ConvNet, self).__init__()
#         # Input image size = 70x320x3

#         self.conv1 = nn.Conv2d(3, 16, 7, 2)  # 32x157
#         self.conv2 = nn.Conv2d(16, 32, 7, 2) # 13x76
#         self.conv3 = nn.Conv2d(32, 64, 7, 2) # 4x35
#         self.pool1 = nn.MaxPool2d(2, 2) # 2x17
#         self.dropout1 = nn.Dropout2d(0.25)
        
#         self.fc1 = nn.Linear(2*17*64, 512)
#         self.dropout2 = nn.Dropout2d(0.25)
#         self.fc2 = nn.Linear(512, 1)
        

#     def forward(self, x):
  
#         x = F.relu(self.conv1(x))
#         x = F.relu(self.conv2(x))
#         x = self.pool1(F.relu(self.conv3(x)))
#         x = self.dropout1(x)

#         x = x.view(-1, 2*17*64)
#         x = F.relu(self.fc1(x))
#         x = self.dropout2(x)
#         x = self.fc2(x)
#         return x

# # Define model
# print("==> Initialize model ...")
# model = ConvNet()
# # print("==> Initialize model done ...")

In [21]:
# Creating model
# CNN7 - 3 convolution layers, 3x3 filters, YUV image


#class ConvNet(nn.Module):
#     def __init__(self):
#         super(ConvNet, self).__init__()
#         # Input image size = 128x128x3
#         # Input image size = 70x320x3

#         self.conv1 = nn.Conv2d(3, 16, 3, padding = 'same')  # 70x320
#         self.pool1 = nn.MaxPool2d(2, 2) # 35x160

#         self.conv2 = nn.Conv2d(16, 32, 3, padding = 'same') # 35x160
#         self.pool2 = nn.MaxPool2d(2, 2) # 17x80

#         self.conv3 = nn.Conv2d(32, 64, 3, padding = 'same') # 17x80
#         self.pool3 = nn.MaxPool2d(2, 2) # 8x40
#         self.dropout1 = nn.Dropout2d(0.25)

        
#         self.fc1 = nn.Linear(8*40*64, 512)
#         self.dropout2 = nn.Dropout2d(0.25)
#         self.fc2 = nn.Linear(512, 1)
        

#     def forward(self, x):
  
#         x = self.pool1(F.relu(self.conv1(x)))
#         x = self.pool2(F.relu(self.conv2(x)))
#         x = self.pool3(F.relu(self.conv3(x)))
#         x = self.dropout1(x)

#         x = x.view(-1, 8*40*64)
#         x = F.relu(self.fc1(x))
#         x = self.dropout2(x)
#         x = self.fc2(x)
#         return x

# # Define model
# print("==> Initialize model ...")
# model = ConvNet()
# print("==> Initialize model done ...")

In [22]:
# # Creating model
# # CNN8 - 4 convolution layers, 3x3 filters, YUV image

# class ConvNet(nn.Module):
#     def __init__(self):
#         super(ConvNet, self).__init__()
#         # Input image size = 70x320x3

#         self.conv1 = nn.Conv2d(3, 16, 3, padding = 'same')  # 70x320
#         self.pool1 = nn.MaxPool2d(2, 2) # 35x160

#         self.conv2 = nn.Conv2d(16, 32, 3, padding = 'same') # 35x160
#         self.pool2 = nn.MaxPool2d(2, 2) # 17x80

#         self.conv3 = nn.Conv2d(32, 64, 3, padding = 'same') # 17x80
#         self.pool3 = nn.MaxPool2d(2, 2) # 8x40
        
#         self.conv4 = nn.Conv2d(64, 64, 3, padding = 'same') # 8x40
#         self.pool4 = nn.MaxPool2d(2, 2) # 4x20
#         self.dropout1 = nn.Dropout2d(0.25)

        
#         self.fc1 = nn.Linear(4*20*64, 512)
#         self.dropout2 = nn.Dropout2d(0.25)
#         self.fc2 = nn.Linear(512, 1)
        

#     def forward(self, x):
  
#         x = self.pool1(F.relu(self.conv1(x)))
#         x = self.pool2(F.relu(self.conv2(x)))
#         x = self.pool3(F.relu(self.conv3(x)))
#         x = self.pool4(F.relu(self.conv4(x)))
#         x = self.dropout1(x)

#         x = x.view(-1, 4*20*64)
#         x = F.relu(self.fc1(x))
#         x = self.dropout2(x)
#         x = self.fc2(x)
#         return x

# # Define model
# print("==> Initialize model ...")
# model = ConvNet()
# print("==> Initialize model done ...")

In [23]:
# Creating model
# CNN9 - 4 convolution layers, 3x3 and 5x5 filters, YUV image

class ConvNet(nn.Module):
    def __init__(self):
        super(ConvNet, self).__init__()
        # Input image size = 70x320x3

        self.conv1 = nn.Conv2d(3, 16, 3, padding = 'same')  # 70x320
        self.pool1 = nn.MaxPool2d(2, 2) # 35x160

        self.conv2 = nn.Conv2d(16, 32, 3, padding = 'same') # 35x160
        self.pool2 = nn.MaxPool2d(2, 2) # 17x80

        self.conv3 = nn.Conv2d(32, 64, 3, padding = 'same') # 17x80
        self.pool3 = nn.MaxPool2d(2, 2) # 8x40
        
        self.conv4 = nn.Conv2d(64, 64, 5, 2) # 2x18, strides=2
        self.pool4 = nn.MaxPool2d(2, 2) # 1x9
        self.dropout1 = nn.Dropout2d(0.25)

        
        self.fc1 = nn.Linear(1*9*64, 256)
        self.dropout2 = nn.Dropout2d(0.25)
        self.fc2 = nn.Linear(256, 1)
        

    def forward(self, x):
  
        x = self.pool1(F.relu(self.conv1(x)))
        x = self.pool2(F.relu(self.conv2(x)))
        x = self.pool3(F.relu(self.conv3(x)))
        x = self.pool4(F.relu(self.conv4(x)))
        x = self.dropout1(x)

        x = x.view(-1, 1*9*64)
        x = F.relu(self.fc1(x))
        x = self.dropout2(x)
        x = self.fc2(x)
        return x

# Define model
print("==> Initialize model ...")
model = ConvNet()
print("==> Initialize model done ...")

==> Initialize model ...
==> Initialize model done ...


In [24]:
print(model)

ConvNet(
  (conv1): Conv2d(3, 16, kernel_size=(3, 3), stride=(1, 1), padding=same)
  (pool1): MaxPool2d(kernel_size=2, stride=2, padding=0, dilation=1, ceil_mode=False)
  (conv2): Conv2d(16, 32, kernel_size=(3, 3), stride=(1, 1), padding=same)
  (pool2): MaxPool2d(kernel_size=2, stride=2, padding=0, dilation=1, ceil_mode=False)
  (conv3): Conv2d(32, 64, kernel_size=(3, 3), stride=(1, 1), padding=same)
  (pool3): MaxPool2d(kernel_size=2, stride=2, padding=0, dilation=1, ceil_mode=False)
  (conv4): Conv2d(64, 64, kernel_size=(5, 5), stride=(2, 2))
  (pool4): MaxPool2d(kernel_size=2, stride=2, padding=0, dilation=1, ceil_mode=False)
  (dropout1): Dropout2d(p=0.25, inplace=False)
  (fc1): Linear(in_features=576, out_features=256, bias=True)
  (dropout2): Dropout2d(p=0.25, inplace=False)
  (fc2): Linear(in_features=256, out_features=1, bias=True)
)


In [25]:
# Calculating parameters and memory

from torchstat import stat
stat(model, (3, 70, 320))

[MAdd]: Dropout2d is not supported!
[Flops]: Dropout2d is not supported!
[Memory]: Dropout2d is not supported!
[MAdd]: Dropout2d is not supported!
[Flops]: Dropout2d is not supported!
[Memory]: Dropout2d is not supported!
      module name  input shape output shape    params memory(MB)           MAdd         Flops  MemRead(B)  MemWrite(B) duration[%]  MemR+W(B)
0           conv1    3  70 320   16  70 320     448.0       1.37   19,353,600.0  10,035,200.0    270592.0    1433600.0      48.50%  1704192.0
1           pool1   16  70 320   16  35 160       0.0       0.34      268,800.0     358,400.0   1433600.0     358400.0       7.23%  1792000.0
2           conv2   16  35 160   32  35 160    4640.0       0.68   51,609,600.0  25,984,000.0    376960.0     716800.0      13.88%  1093760.0
3           pool2   32  35 160   32  17  80       0.0       0.17      130,560.0     179,200.0    716800.0     174080.0       4.42%   890880.0
4           conv3   32  17  80   64  17  80   18496.0       0.33   5

In [25]:
# # Creating model
# # CNN10 - 4 convolution layers, 3x3 and 5x5 filters, 2FCs, YUV image

# class ConvNet(nn.Module):
#     def __init__(self):
#         super(ConvNet, self).__init__()
#         # Input image size = 70x320x3

#         self.conv1 = nn.Conv2d(3, 16, 3, padding = 'same')  # 70x320
#         self.pool1 = nn.MaxPool2d(2, 2) # 35x160

#         self.conv2 = nn.Conv2d(16, 32, 3, padding = 'same') # 35x160
#         self.pool2 = nn.MaxPool2d(2, 2) # 17x80

#         self.conv3 = nn.Conv2d(32, 64, 3, padding = 'same') # 17x80
#         self.pool3 = nn.MaxPool2d(2, 2) # 8x40
        
#         self.conv4 = nn.Conv2d(64, 64, 5, 2) # 2x18, strides=2
#         self.pool4 = nn.MaxPool2d(2, 2) # 1x9
#         self.dropout1 = nn.Dropout2d(0.25)

        
#         self.fc1 = nn.Linear(1*9*64, 256)
#         self.fc2 = nn.Linear(256, 64)
#         self.dropout2 = nn.Dropout2d(0.25)
#         self.fc3 = nn.Linear(64, 1)
        

#     def forward(self, x):
  
#         x = self.pool1(F.relu(self.conv1(x)))
#         x = self.pool2(F.relu(self.conv2(x)))
#         x = self.pool3(F.relu(self.conv3(x)))
#         x = self.pool4(F.relu(self.conv4(x)))
#         x = self.dropout1(x)

#         x = x.view(-1, 1*9*64)
#         x = F.relu(self.fc1(x))
#         x = F.relu(self.fc2(x))
#         x = self.dropout2(x)
#         x = self.fc3(x)
#         return x


# # Define model
# print("==> Initialize model ...")
# model = ConvNet()
# print("==> Initialize model done ...")

==> Initialize model ...
==> Initialize model done ...


In [26]:
# Creating model
# CNN11 - 4 convolution layers, 5x5 and 3x3 filters, 2FCs, YUV image

# class ConvNet(nn.Module):
#     def __init__(self):
#         super(ConvNet, self).__init__()
#         # Input image size = 70x320x3

#         self.conv1 = nn.Conv2d(3, 16, 5, 2)  # 33x158
#         self.pool1 = nn.MaxPool2d(2, 2) # 16x79

#         self.conv2 = nn.Conv2d(16, 32, 3, padding = 'same') # 16x79
#         self.pool2 = nn.MaxPool2d(2, 2) # 8x39

#         self.conv3 = nn.Conv2d(32, 64, 3, padding = 'same') # 8x39
#         self.pool3 = nn.MaxPool2d(2, 2) # 4x19
        
#         self.conv4 = nn.Conv2d(64, 64, 3, padding = 'same') # 4x19
#         self.pool4 = nn.MaxPool2d(2, 2) # 2x9
#         self.dropout1 = nn.Dropout2d(0.25)

        
#         self.fc1 = nn.Linear(2*9*64, 256)
#         self.fc2 = nn.Linear(256, 64)
#         self.dropout2 = nn.Dropout2d(0.25)
#         self.fc3 = nn.Linear(64, 1)
        

#     def forward(self, x):
  
#         x = self.pool1(F.relu(self.conv1(x)))
#         x = self.pool2(F.relu(self.conv2(x)))
#         x = self.pool3(F.relu(self.conv3(x)))
#         x = self.pool4(F.relu(self.conv4(x)))
#         x = self.dropout1(x)

#         x = x.view(-1, 2*9*64)
#         x = F.relu(self.fc1(x))
#         x = F.relu(self.fc2(x))
#         x = self.dropout2(x)
#         x = self.fc3(x)
#         return x


# # Define model
# print("==> Initialize model ...")
# model = ConvNet()
# print("==> Initialize model done ...")

In [27]:
# Creating model
# CNN12 - 4 convolution layers, 5x5 and 3x3 filters, 2FCs, YUV image

# class ConvNet(nn.Module):
#     def __init__(self):
#         super(ConvNet, self).__init__()
#         # Input image size = 70x320x3

#         self.conv1 = nn.Conv2d(3, 16, 5, 2)  # 33x158
#         self.conv2 = nn.Conv2d(16, 32, 5, 2) # 15x77
#         self.pool1 = nn.MaxPool2d(2, 2) # 7x38

#         self.conv3 = nn.Conv2d(32, 64, 3) # 5x36       
#         self.conv4 = nn.Conv2d(64, 64, 3) # 3x34
#         self.pool2 = nn.MaxPool2d(2, 2) # 1x17
#         self.dropout1 = nn.Dropout2d(0.25)

        
#         self.fc1 = nn.Linear(1*17*64, 256)
#         self.fc2 = nn.Linear(256, 64)
#         self.dropout2 = nn.Dropout2d(0.25)
#         self.fc3 = nn.Linear(64, 1)
        

#     def forward(self, x):
  
#         x = F.relu(self.conv1(x))
#         x = self.pool1(F.relu(self.conv2(x)))
#         x = F.relu(self.conv3(x))
#         x = self.pool2(F.relu(self.conv4(x)))
#         x = self.dropout1(x)

#         x = x.view(-1, 1*17*64)
#         x = F.relu(self.fc1(x))
#         x = F.relu(self.fc2(x))
#         x = self.dropout2(x)
#         x = self.fc3(x)
#         return x


# # Define model
# print("==> Initialize model ...")
# model = ConvNet()
# print("==> Initialize model done ...")

In [28]:
# Define optimizer and criterion
optimizer = optim.Adam(model.parameters(),
                       lr=lr,
                       weight_decay=weight_decay)
criterion = nn.MSELoss()

In [29]:
# learning rate scheduler
scheduler = MultiStepLR(optimizer, milestones=[30, 50], gamma=0.1)

# transfer to gpu
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
print(device)

cuda


In [38]:
# Resuming training

if resume:
    print("==> Loading checkpoint ...")
    checkpoint = torch.load("../model2_CNN10_YUV-30.h5",
                            map_location=lambda storage, loc: storage)
    start_epoch = checkpoint['epoch']
    model.load_state_dict(checkpoint['state_dict'])
    optimizer.load_state_dict(checkpoint['optimizer'])
    scheduler.load_state_dict(checkpoint['scheduler'])

In [39]:
# Model training

class Trainer(object):
    """Trainer."""

    def __init__(self,
                 ckptroot,
                 model,
                 device,
                 epochs,
                 criterion,
                 optimizer,
                 scheduler,
                 start_epoch,
                 trainloader,
                 validationloader):
        """Self-Driving car Trainer.

        Args:
            model:
            device:
            epochs:
            criterion:
            optimizer:
            start_epoch:
            trainloader:
            validationloader:

        """
        super(Trainer, self).__init__()

        self.model = model
        self.device = device
        self.epochs = epochs
        self.ckptroot = ckptroot
        self.criterion = criterion
        self.optimizer = optimizer
        self.scheduler = scheduler
        self.start_epoch = start_epoch
        self.trainloader = trainloader
        self.validationloader = validationloader

    def train(self):
        """Training process."""
        self.model.to(self.device)
        Training_loss = []
        Validation_loss = []
        epochs = []

        for epoch in range(self.start_epoch, self.epochs + self.start_epoch):
            epochs.append(epoch)
            self.scheduler.step()
            
            # Training
            train_loss = 0.0
            self.model.train()
            #print("Before train loading")

            for local_batch, (centers, lefts, rights) in enumerate(self.trainloader):
                #print("After train loading")
                # Transfer to GPU
                centers, lefts, rights = toDevice(centers, self.device), toDevice(
                    lefts, self.device), toDevice(rights, self.device)

                # Model computations
                self.optimizer.zero_grad()
                datas = [centers, lefts, rights]
                for data in datas:
                    imgs, angles = data
                    # print("training image: ", imgs.shape)
                    outputs = self.model(imgs)
                    loss = self.criterion(outputs, angles.unsqueeze(1))
                    loss.backward()
                    self.optimizer.step()

                    train_loss += loss.data.item()

                if local_batch % 100 == 0:

                    print("Training Epoch: {} | Loss: {}".format(epoch, train_loss / (local_batch + 1)))
                    Training_loss.append(train_loss)


            # Validation
            self.model.eval()
            valid_loss = 0
            with torch.set_grad_enabled(False):
                for local_batch, (centers, lefts, rights) in enumerate(self.validationloader):
                    # Transfer to GPU
                    centers, lefts, rights = toDevice(centers, self.device), toDevice(
                        lefts, self.device), toDevice(rights, self.device)

                    # Model computations
                    self.optimizer.zero_grad()
                    datas = [centers, lefts, rights]
                    predicts = []
                    actuals = []
                    for data in datas:
                        imgs, angles = data
                        #print('Angles:', angles)
                        actuals.append(angles.cpu().squeeze())
                        outputs = self.model(imgs)
                        #print('Outputs: ', outputs.reshape(-1))
                        #x = pd.DataFrame(angles.cpu().squeeze(), columns = ['Actual angles'])
                        #y = pd.DataFrame(outputs.reshape(-1).cpu().squeeze(), columns = ['Predicted angles'])
                        #print(pd.concat([x,y], axis =1))
                        loss = self.criterion(outputs, angles.unsqueeze(1))
                        predicts.append(outputs.reshape(-1).cpu().squeeze())

                        valid_loss += loss.data.item()

                    if local_batch % 100 == 0:
                        print("Validation Loss: {}".format(valid_loss / (local_batch + 1)))
                        Validation_loss.append(valid_loss)
                        

            
            #plt.plot(Validation_loss,epochs)
            #print('actuals shape:', len(actuals))
            #print('predicts shape:', len(predicts))

            x = pd.DataFrame(actuals[1], columns = ['Actual Angles'])
            y = pd.DataFrame(predicts[1], columns = ['Predicted Angles'])
            #print(x)
            #print(y)
            print(pd.concat([x,y], axis=1))
            print('MSE:',mean_squared_error(actuals[1],predicts[1]))
            print()
            # Save model
                        
#             if epoch % 5 == 0 or epoch == self.epochs + self.start_epoch - 1:

#                 state = {
#                     'epoch': epoch + 1,
#                     'state_dict': self.model.state_dict(),
#                     'optimizer': self.optimizer.state_dict(),
#                     'scheduler': self.scheduler.state_dict(),
#                 }

#                 self.save_checkpoint(state)



            state = {
                'epoch': epoch + 1,
                'state_dict': self.model.state_dict(),
                'optimizer': self.optimizer.state_dict(),
                'scheduler': self.scheduler.state_dict(),
            }

            self.save_checkpoint(state)

    def save_checkpoint(self, state):
        """Save checkpoint."""
        print("==> Save checkpoint ...")
        if not os.path.exists(self.ckptroot):
            os.makedirs(self.ckptroot)

        torch.save(state, self.ckptroot + 'model2_CNN10_YUV-{}.h5'.format(state['epoch']))

In [40]:

print("==> Start training ...")
trainer = Trainer(ckptroot,
                  model,
                  device,
                  epochs,
                  criterion,
                  optimizer,
                  scheduler,
                  start_epoch,
                  trainloader,
                  validationloader)
trainer.train()

==> Start training ...


C:\Users\Lakshmikar Reddy\anaconda3\envs\gpu\lib\site-packages\torch\nn\functional.py:1331: UserWarning: dropout2d: Received a 2-D input to dropout2d, which is deprecated and will result in an error in a future release. To retain the behavior and silence this warning, please use dropout instead. Note that dropout2d exists to provide channel-wise dropout on inputs with 2 spatial dimensions, a channel dimension, and an optional batch dimension (i.e. 3D or 4D inputs).
  warnings.warn(warn_msg)


Training Epoch: 0 | Loss: 0.24815470352768898
Training Epoch: 0 | Loss: 0.26865641206576685
Training Epoch: 0 | Loss: 0.2738899273695934
Training Epoch: 0 | Loss: 0.2762955454212189
Training Epoch: 0 | Loss: 0.27610485501438753
Training Epoch: 0 | Loss: 0.2796681000518347
Training Epoch: 0 | Loss: 0.27858361703174783
Training Epoch: 0 | Loss: 0.27840109794970586
Training Epoch: 0 | Loss: 0.2786810656569982
Training Epoch: 0 | Loss: 0.2790290174652284
Training Epoch: 0 | Loss: 0.27874585562995025
Training Epoch: 0 | Loss: 0.2785858698662956
Training Epoch: 0 | Loss: 0.2782615296617088
Training Epoch: 0 | Loss: 0.2789996483310954
Validation Loss: 0.29314326494932175
    Actual Angles  Predicted Angles
0            0.00          0.181924
1            0.40          0.140762
2            0.05         -0.349173
3           -0.40         -0.315354
4            0.40          0.367883
5           -0.40         -0.431894
6           -0.40         -0.034802
7            0.40          0.160928
8  

C:\Users\Lakshmikar Reddy\anaconda3\envs\gpu\lib\site-packages\torch\nn\functional.py:1331: UserWarning: dropout2d: Received a 2-D input to dropout2d, which is deprecated and will result in an error in a future release. To retain the behavior and silence this warning, please use dropout instead. Note that dropout2d exists to provide channel-wise dropout on inputs with 2 spatial dimensions, a channel dimension, and an optional batch dimension (i.e. 3D or 4D inputs).
  warnings.warn(warn_msg)


Training Epoch: 1 | Loss: 0.19705524295568466
Training Epoch: 1 | Loss: 0.26546599205103844
Training Epoch: 1 | Loss: 0.26420496908289876
Training Epoch: 1 | Loss: 0.2670305301481703
Training Epoch: 1 | Loss: 0.26932243314430004
Training Epoch: 1 | Loss: 0.26953937340074197
Training Epoch: 1 | Loss: 0.27098355856964274
Training Epoch: 1 | Loss: 0.2723920789843534
Training Epoch: 1 | Loss: 0.2720308697941747
Training Epoch: 1 | Loss: 0.2724969223985132
Training Epoch: 1 | Loss: 0.2730293536929043
Training Epoch: 1 | Loss: 0.2737912609614939
Training Epoch: 1 | Loss: 0.27510799809694786
Training Epoch: 1 | Loss: 0.2757804834818629
Validation Loss: 0.18306756392121315
    Actual Angles  Predicted Angles
0            0.60         -0.036459
1            0.40          0.211845
2            0.40          0.089062
3            0.40          0.393315
4           -0.40         -0.305703
5            0.40          0.148402
6            0.40          0.101375
7           -0.75         -0.523649
8 

...

Training Epoch: 7 | Loss: 0.3338640108704567
Training Epoch: 7 | Loss: 0.2522893071285274
Training Epoch: 7 | Loss: 0.25246861198018145
Training Epoch: 7 | Loss: 0.25451842945477493
Training Epoch: 7 | Loss: 0.2550532904330483
Training Epoch: 7 | Loss: 0.2559753583443022
Training Epoch: 7 | Loss: 0.2575274114303303
Training Epoch: 7 | Loss: 0.25752740234306637
Training Epoch: 7 | Loss: 0.25674955168513397
Training Epoch: 7 | Loss: 0.2569564184283509
Training Epoch: 7 | Loss: 0.25758796367984077
Training Epoch: 7 | Loss: 0.2584254981731686
Training Epoch: 7 | Loss: 0.2582095854734997
Training Epoch: 7 | Loss: 0.25772454284223156
Validation Loss: 0.2795211225748062
    Actual Angles  Predicted Angles
0             0.4          0.115153
1            -0.4         -0.406224
2            -0.2         -0.063061
3             0.4          0.066443
4             0.4          0.363991
5            -0.4         -0.495766
6            -0.4         -0.387823
7             0.4          0.439620
8   

C:\Users\Lakshmikar Reddy\anaconda3\envs\gpu\lib\site-packages\torch\nn\functional.py:1331: UserWarning: dropout2d: Received a 2-D input to dropout2d, which is deprecated and will result in an error in a future release. To retain the behavior and silence this warning, please use dropout instead. Note that dropout2d exists to provide channel-wise dropout on inputs with 2 spatial dimensions, a channel dimension, and an optional batch dimension (i.e. 3D or 4D inputs).
  warnings.warn(warn_msg)


Training Epoch: 8 | Loss: 0.24499327689409256
Training Epoch: 8 | Loss: 0.2574671365800175
Training Epoch: 8 | Loss: 0.25692689686605885
Training Epoch: 8 | Loss: 0.2539384611804917
Training Epoch: 8 | Loss: 0.2542176183238811
Training Epoch: 8 | Loss: 0.25214214726888434
Training Epoch: 8 | Loss: 0.25006238381206347
Training Epoch: 8 | Loss: 0.250842076096519
Training Epoch: 8 | Loss: 0.2517654741968908
Training Epoch: 8 | Loss: 0.2520468036047742
Training Epoch: 8 | Loss: 0.25288651851503613
Training Epoch: 8 | Loss: 0.2533880049163149
Training Epoch: 8 | Loss: 0.25327888998954395
Training Epoch: 8 | Loss: 0.2536947560159462
Validation Loss: 0.24535883218050003
    Actual Angles  Predicted Angles
0            0.60         -0.225192
1           -0.40         -0.276227
2            0.40          0.445904
3            0.40          0.391239
4            0.40          0.210898
5            0.40         -0.007980
6           -0.40          0.009559
7            0.35          0.038927
8   

C:\Users\Lakshmikar Reddy\anaconda3\envs\gpu\lib\site-packages\torch\nn\functional.py:1331: UserWarning: dropout2d: Received a 2-D input to dropout2d, which is deprecated and will result in an error in a future release. To retain the behavior and silence this warning, please use dropout instead. Note that dropout2d exists to provide channel-wise dropout on inputs with 2 spatial dimensions, a channel dimension, and an optional batch dimension (i.e. 3D or 4D inputs).
  warnings.warn(warn_msg)


Training Epoch: 9 | Loss: 0.2293303906917572
Training Epoch: 9 | Loss: 0.23459088317312227
Training Epoch: 9 | Loss: 0.2389694579427515
Training Epoch: 9 | Loss: 0.2391717255152638
Training Epoch: 9 | Loss: 0.2401826148875931
Training Epoch: 9 | Loss: 0.24119076896421685
Training Epoch: 9 | Loss: 0.24095386203483812
Training Epoch: 9 | Loss: 0.24016347383487752
Training Epoch: 9 | Loss: 0.24017457195909953
Training Epoch: 9 | Loss: 0.23971356755199033
Training Epoch: 9 | Loss: 0.23911461348996146
Training Epoch: 9 | Loss: 0.23876904050729067
Training Epoch: 9 | Loss: 0.23782186687448614
Training Epoch: 9 | Loss: 0.23715455577043754
Validation Loss: 0.21211335808038712
    Actual Angles  Predicted Angles
0            -0.4         -0.357181
1            -0.3         -0.176965
2            -0.4         -0.086708
3             0.4         -0.001373
4            -0.4          0.049965
5             0.4         -0.005352
6            -0.6          0.307031
7            -0.4         -0.198034

In [54]:
state = {
        'model': model.module if device == 'cuda' else model,
        }

# torch.save(state, 'model' + model_name+ '.h5')
torch.save(state, 'model2_CNN9.h5')

In [33]:
# Loading weights for Testing

state = torch.load("./model2_CNN10_YUV-19.h5")
model.load_state_dict(state['state_dict'])
#optimizer.load_state_dict(state['optimizer'])

<All keys matched successfully>

In [34]:
# Testing to calculate MSE

with torch.set_grad_enabled(False):
    #print("Hi")
    for local_batch, (centers, lefts, rights) in enumerate(testloader):
        #print("Hi all")
        centers, lefts, rights = toDevice(centers, device), toDevice(lefts, device), toDevice(rights, device) # Transfer to GPU

        # Model computations
        optimizer.zero_grad()
        datas = [centers, lefts, rights]
        predicts = []
        actuals = []
        for data in datas:
            imgs, angles = data
            actuals.append(angles.cpu().squeeze())
            outputs = model(imgs)
            loss = criterion(outputs, angles.unsqueeze(1))
            predicts.append(outputs.reshape(-1).cpu().squeeze())
            
x = pd.DataFrame(actuals[0], columns = ['Actual Angles'])
y = pd.DataFrame(predicts[0], columns = ['Predicted Angles'])
print(pd.concat([x,y], axis=1))
print('MSE:',mean_squared_error(actuals[0],predicts[0]))
#print(actuals)

C:\Users\Lakshmikar Reddy\anaconda3\envs\gpu\lib\site-packages\torch\nn\functional.py:1331: UserWarning: dropout2d: Received a 2-D input to dropout2d, which is deprecated and will result in an error in a future release. To retain the behavior and silence this warning, please use dropout instead. Note that dropout2d exists to provide channel-wise dropout on inputs with 2 spatial dimensions, a channel dimension, and an optional batch dimension (i.e. 3D or 4D inputs).
  warnings.warn(warn_msg)


    Actual Angles  Predicted Angles
0       -0.340585         -0.122153
1        0.000000         -0.164918
2        0.000000         -0.176276
3       -0.071721         -0.137472
4       -0.417630         -0.102983
5       -0.116112         -0.132684
6        0.000000         -0.121567
7        0.000000         -0.114900
8        0.000000         -0.105712
9        0.000000         -0.191875
10       0.000000         -0.143677
11       0.000000         -0.179033
12       0.000000         -0.212254
13      -0.071269         -0.207851
14      -0.391136         -0.189488
15      -0.279243         -0.191007
16       0.000000         -0.184803
17       0.000000         -0.222327
18       0.000000         -0.215403
19       0.000000         -0.226560
20       0.000000         -0.237701
21       0.000000         -0.236624
22       0.000000         -0.227489
23       0.000000         -0.247275
24       0.000000         -0.258622
25       0.000000         -0.264047
26       0.000000         -0